In [1]:
import json
import csv
import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import GetOldTweets3 as got
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import numpy as np

In [2]:
df=pd.read_csv('../../Datasets/Twitter/Datasets with likes/final_au_withlikes.csv')
df

,Unnamed: 0,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score
0,0,"@rsprasad @IndiaPostOffice \nSir,\nRegarding U...",ranjeeta,1157270873314693120,2019-08-02 12:44:11,[],ranjeeta21,0,0,NaN,-0.2824,Negative,2
1,1,EduKhoj connected another student with School ...,EduKhoj,1156079145270439936,2019-07-30 05:48:41,['Bengaluru'],EduKhoj,0,0,NaN,0.0000,Neutral,3
2,2,Bangalore Blockchain Summit happening at Allia...,WWCode Blockchain,1155059429055709184,2019-07-27 10:16:42,"['blockchain', 'summit', 'allianceuniversity',...",WWCodeBlock,0,1,NaN,0.4588,Positive,4
3,3,Alliance University in partnership with KPI Co...,TotalBitcoinInternational,1154961017325203456,2019-07-27 03:45:39,['Blockchain'],bitcoin_total,0,0,NaN,0.0000,Neutral,3
4,4,"@rsprasad @IndiaPostOffice \nSir,\nRegarding U...",ranjeeta,1157270873314693120,2019-08-02 12:44:11,[],ranjeeta21,0,0,NaN,-0.2824,Negative,2
5,5,EduKhoj connected another student with School ...,EduKhoj,1156079145270439936,2019-07-30 05:48:41,['Bengaluru'],EduKhoj,0,0,NaN,0.0000,Neutral,3
6,6,Bangalore Blockchain Summit happening at Allia...,WWCode Blockchain,1155059429055709184,2019-07-27 10:16:42,"['blockchain', 'summit', 'allianceuniversity',...",WWCodeBlock,0,1,NaN,0.4588,Positive,4
7,7,Alliance University in partnership with KPI Co...,TotalBitcoinInternational,1154961017325203456,2019-07-27 03:45:39,['Blockchain'],bitcoin_total,0,0,NaN,0.0000,Neutral,3
8,8,On my way to our second @CircleU_eu student s...,Youcef KADRI,1202917705046736896,2019-12-06 11:48:24,[],Youcef__KADRI,0,8,NaN,0.8614,Positive,5
9,9,On my way to our second @CircleU_eu student s...,Youcef KADRI,1202917705046736896,2019-12-06 11:48:24,[],Youcef__KADRI,0,8,NaN,0.8614,Positive,5


In [3]:
df=df.drop(['Unnamed: 0'],axis=1)

In [4]:
def normalize(feature):
    max_value = df[feature].max()
    min_value = df[feature].min()
    result = (df[feature] - min_value) / (max_value - min_value)
    return result

In [5]:
df['retweets']=df['retweets'].astype(float)
df['likes']=df['likes'].astype(float)

In [6]:
df['retweets_n']=normalize('retweets')
df['likes_n']=normalize('likes')
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n
0,"@rsprasad @IndiaPostOffice \nSir,\nRegarding U...",ranjeeta,1157270873314693120,2019-08-02 12:44:11,[],ranjeeta21,0.0,0.0,NaN,-0.2824,Negative,2,0.000000,0.000000
1,EduKhoj connected another student with School ...,EduKhoj,1156079145270439936,2019-07-30 05:48:41,['Bengaluru'],EduKhoj,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000
2,Bangalore Blockchain Summit happening at Allia...,WWCode Blockchain,1155059429055709184,2019-07-27 10:16:42,"['blockchain', 'summit', 'allianceuniversity',...",WWCodeBlock,0.0,1.0,NaN,0.4588,Positive,4,0.000000,0.008403
3,Alliance University in partnership with KPI Co...,TotalBitcoinInternational,1154961017325203456,2019-07-27 03:45:39,['Blockchain'],bitcoin_total,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000
4,"@rsprasad @IndiaPostOffice \nSir,\nRegarding U...",ranjeeta,1157270873314693120,2019-08-02 12:44:11,[],ranjeeta21,0.0,0.0,NaN,-0.2824,Negative,2,0.000000,0.000000
5,EduKhoj connected another student with School ...,EduKhoj,1156079145270439936,2019-07-30 05:48:41,['Bengaluru'],EduKhoj,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000
6,Bangalore Blockchain Summit happening at Allia...,WWCode Blockchain,1155059429055709184,2019-07-27 10:16:42,"['blockchain', 'summit', 'allianceuniversity',...",WWCodeBlock,0.0,1.0,NaN,0.4588,Positive,4,0.000000,0.008403
7,Alliance University in partnership with KPI Co...,TotalBitcoinInternational,1154961017325203456,2019-07-27 03:45:39,['Blockchain'],bitcoin_total,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000
8,On my way to our second @CircleU_eu student s...,Youcef KADRI,1202917705046736896,2019-12-06 11:48:24,[],Youcef__KADRI,0.0,8.0,NaN,0.8614,Positive,5,0.000000,0.067227
9,On my way to our second @CircleU_eu student s...,Youcef KADRI,1202917705046736896,2019-12-06 11:48:24,[],Youcef__KADRI,0.0,8.0,NaN,0.8614,Positive,5,0.000000,0.067227


In [7]:
df['engagement_score']=60 * df['retweets_n'] + 40 * df['likes_n']

In [8]:
def clean_tweet(tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split()) #(@[A-Za-z0-9]+)|

In [9]:
df['clean_tweets']=df['text'].map(clean_tweet)
df['text']=df['text'].map(str.lower)

In [10]:
from nltk.probability import FreqDist
from nltk import pos_tag, word_tokenize
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt

from sklearn.manifold import MDS
from sklearn.metrics import euclidean_distances

In [11]:
df["clean_tweets"] = df["clean_tweets"].astype(str)
df["tweet_tokens"] = df["clean_tweets"].apply(lambda each_post: word_tokenize(re.sub(r'[^\w\s]',' ',each_post.lower())))
df["tweet_tokens"] = df["tweet_tokens"].apply(lambda list_of_words: [x for x in list_of_words if x not in stopwords])
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens
0,"@rsprasad @indiapostoffice \nsir,\nregarding u...",ranjeeta,1157270873314693120,2019-08-02 12:44:11,[],ranjeeta21,0.0,0.0,NaN,-0.2824,Negative,2,0.000000,0.000000,0.000000,"@rsprasad @IndiaPostOffice Sir, Regarding Urge...","[rsprasad, indiapostoffice, sir, regarding, ur..."
1,edukhoj connected another student with school ...,EduKhoj,1156079145270439936,2019-07-30 05:48:41,['Bengaluru'],EduKhoj,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,EduKhoj connected another student with School ...,"[edukhoj, connected, another, student, school,..."
2,bangalore blockchain summit happening at allia...,WWCode Blockchain,1155059429055709184,2019-07-27 10:16:42,"['blockchain', 'summit', 'allianceuniversity',...",WWCodeBlock,0.0,1.0,NaN,0.4588,Positive,4,0.000000,0.008403,0.336134,Bangalore Blockchain Summit happening at Allia...,"[bangalore, blockchain, summit, happening, all..."
3,alliance university in partnership with kpi co...,TotalBitcoinInternational,1154961017325203456,2019-07-27 03:45:39,['Blockchain'],bitcoin_total,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,Alliance University in partnership with KPI Co...,"[alliance, university, partnership, kpi, consu..."
4,"@rsprasad @indiapostoffice \nsir,\nregarding u...",ranjeeta,1157270873314693120,2019-08-02 12:44:11,[],ranjeeta21,0.0,0.0,NaN,-0.2824,Negative,2,0.000000,0.000000,0.000000,"@rsprasad @IndiaPostOffice Sir, Regarding Urge...","[rsprasad, indiapostoffice, sir, regarding, ur..."
5,edukhoj connected another student with school ...,EduKhoj,1156079145270439936,2019-07-30 05:48:41,['Bengaluru'],EduKhoj,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,EduKhoj connected another student with School ...,"[edukhoj, connected, another, student, school,..."
6,bangalore blockchain summit happening at allia...,WWCode Blockchain,1155059429055709184,2019-07-27 10:16:42,"['blockchain', 'summit', 'allianceuniversity',...",WWCodeBlock,0.0,1.0,NaN,0.4588,Positive,4,0.000000,0.008403,0.336134,Bangalore Blockchain Summit happening at Allia...,"[bangalore, blockchain, summit, happening, all..."
7,alliance university in partnership with kpi co...,TotalBitcoinInternational,1154961017325203456,2019-07-27 03:45:39,['Blockchain'],bitcoin_total,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,Alliance University in partnership with KPI Co...,"[alliance, university, partnership, kpi, consu..."
8,on my way to our second @circleu_eu student s...,Youcef KADRI,1202917705046736896,2019-12-06 11:48:24,[],Youcef__KADRI,0.0,8.0,NaN,0.8614,Positive,5,0.000000,0.067227,2.689076,On my way to our second @CircleU_eu student se...,"[way, second, circleu_eu, student, seminar, lo..."
9,on my way to our second @circleu_eu student s...,Youcef KADRI,1202917705046736896,2019-12-06 11:48:24,[],Youcef__KADRI,0.0,8.0,NaN,0.8614,Positive,5,0.000000,0.067227,2.689076,On my way to our second @CircleU_eu student se...,"[way, second, circleu_eu, student, seminar, lo..."


In [12]:
nltk.download('averaged_perceptron_tagger')
df['pos']=df['tweet_tokens'].apply(nltk.tag.pos_tag)
df

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Manam/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos
0,"@rsprasad @indiapostoffice \nsir,\nregarding u...",ranjeeta,1157270873314693120,2019-08-02 12:44:11,[],ranjeeta21,0.0,0.0,NaN,-0.2824,Negative,2,0.000000,0.000000,0.000000,"@rsprasad @IndiaPostOffice Sir, Regarding Urge...","[rsprasad, indiapostoffice, sir, regarding, ur...","[(rsprasad, JJ), (indiapostoffice, NN), (sir, ..."
1,edukhoj connected another student with school ...,EduKhoj,1156079145270439936,2019-07-30 05:48:41,['Bengaluru'],EduKhoj,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,EduKhoj connected another student with School ...,"[edukhoj, connected, another, student, school,...","[(edukhoj, RB), (connected, VBN), (another, DT..."
2,bangalore blockchain summit happening at allia...,WWCode Blockchain,1155059429055709184,2019-07-27 10:16:42,"['blockchain', 'summit', 'allianceuniversity',...",WWCodeBlock,0.0,1.0,NaN,0.4588,Positive,4,0.000000,0.008403,0.336134,Bangalore Blockchain Summit happening at Allia...,"[bangalore, blockchain, summit, happening, all...","[(bangalore, RB), (blockchain, JJ), (summit, N..."
3,alliance university in partnership with kpi co...,TotalBitcoinInternational,1154961017325203456,2019-07-27 03:45:39,['Blockchain'],bitcoin_total,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,Alliance University in partnership with KPI Co...,"[alliance, university, partnership, kpi, consu...","[(alliance, NN), (university, NN), (partnershi..."
4,"@rsprasad @indiapostoffice \nsir,\nregarding u...",ranjeeta,1157270873314693120,2019-08-02 12:44:11,[],ranjeeta21,0.0,0.0,NaN,-0.2824,Negative,2,0.000000,0.000000,0.000000,"@rsprasad @IndiaPostOffice Sir, Regarding Urge...","[rsprasad, indiapostoffice, sir, regarding, ur...","[(rsprasad, JJ), (indiapostoffice, NN), (sir, ..."
5,edukhoj connected another student with school ...,EduKhoj,1156079145270439936,2019-07-30 05:48:41,['Bengaluru'],EduKhoj,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,EduKhoj connected another student with School ...,"[edukhoj, connected, another, student, school,...","[(edukhoj, RB), (connected, VBN), (another, DT..."
6,bangalore blockchain summit happening at allia...,WWCode Blockchain,1155059429055709184,2019-07-27 10:16:42,"['blockchain', 'summit', 'allianceuniversity',...",WWCodeBlock,0.0,1.0,NaN,0.4588,Positive,4,0.000000,0.008403,0.336134,Bangalore Blockchain Summit happening at Allia...,"[bangalore, blockchain, summit, happening, all...","[(bangalore, RB), (blockchain, JJ), (summit, N..."
7,alliance university in partnership with kpi co...,TotalBitcoinInternational,1154961017325203456,2019-07-27 03:45:39,['Blockchain'],bitcoin_total,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,Alliance University in partnership with KPI Co...,"[alliance, university, partnership, kpi, consu...","[(alliance, NN), (university, NN), (partnershi..."
8,on my way to our second @circleu_eu student s...,Youcef KADRI,1202917705046736896,2019-12-06 11:48:24,[],Youcef__KADRI,0.0,8.0,NaN,0.8614,Positive,5,0.000000,0.067227,2.689076,On my way to our second @CircleU_eu student se...,"[way, second, circleu_eu, student, seminar, lo...","[(way, NN), (second, JJ), (circleu_eu, JJ), (s..."
9,on my way to our second @circleu_eu student s...,Youcef KADRI,1202917705046736896,2019-12-06 11:48:24,[],Youcef__KADRI,0.0,8.0,NaN,0.8614,Positive,5,0.000000,0.067227,2.689076,On my way to our second @CircleU_eu student se...,"[way, second, circleu_eu, student, seminar, lo...","[(way, NN), (second, JJ), (circleu_eu, JJ), (s..."


In [13]:
# Extract the list of nouns
df['nouns']=df['pos'].apply(lambda pos_list: [x[0] for x in pos_list if x[1].startswith('NN')])
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns
0,"@rsprasad @indiapostoffice \nsir,\nregarding u...",ranjeeta,1157270873314693120,2019-08-02 12:44:11,[],ranjeeta21,0.0,0.0,NaN,-0.2824,Negative,2,0.000000,0.000000,0.000000,"@rsprasad @IndiaPostOffice Sir, Regarding Urge...","[rsprasad, indiapostoffice, sir, regarding, ur...","[(rsprasad, JJ), (indiapostoffice, NN), (sir, ...","[indiapostoffice, sir, delivery, parcel, cu336..."
1,edukhoj connected another student with school ...,EduKhoj,1156079145270439936,2019-07-30 05:48:41,['Bengaluru'],EduKhoj,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,EduKhoj connected another student with School ...,"[edukhoj, connected, another, student, school,...","[(edukhoj, RB), (connected, VBN), (another, DT...","[student, school, law, alliance, university, b..."
2,bangalore blockchain summit happening at allia...,WWCode Blockchain,1155059429055709184,2019-07-27 10:16:42,"['blockchain', 'summit', 'allianceuniversity',...",WWCodeBlock,0.0,1.0,NaN,0.4588,Positive,4,0.000000,0.008403,0.336134,Bangalore Blockchain Summit happening at Allia...,"[bangalore, blockchain, summit, happening, all...","[(bangalore, RB), (blockchain, JJ), (summit, N...","[summit, alliance, university, panel, discussi..."
3,alliance university in partnership with kpi co...,TotalBitcoinInternational,1154961017325203456,2019-07-27 03:45:39,['Blockchain'],bitcoin_total,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,Alliance University in partnership with KPI Co...,"[alliance, university, partnership, kpi, consu...","[(alliance, NN), (university, NN), (partnershi...","[alliance, university, partnership, kpi, summi..."
4,"@rsprasad @indiapostoffice \nsir,\nregarding u...",ranjeeta,1157270873314693120,2019-08-02 12:44:11,[],ranjeeta21,0.0,0.0,NaN,-0.2824,Negative,2,0.000000,0.000000,0.000000,"@rsprasad @IndiaPostOffice Sir, Regarding Urge...","[rsprasad, indiapostoffice, sir, regarding, ur...","[(rsprasad, JJ), (indiapostoffice, NN), (sir, ...","[indiapostoffice, sir, delivery, parcel, cu336..."
5,edukhoj connected another student with school ...,EduKhoj,1156079145270439936,2019-07-30 05:48:41,['Bengaluru'],EduKhoj,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,EduKhoj connected another student with School ...,"[edukhoj, connected, another, student, school,...","[(edukhoj, RB), (connected, VBN), (another, DT...","[student, school, law, alliance, university, b..."
6,bangalore blockchain summit happening at allia...,WWCode Blockchain,1155059429055709184,2019-07-27 10:16:42,"['blockchain', 'summit', 'allianceuniversity',...",WWCodeBlock,0.0,1.0,NaN,0.4588,Positive,4,0.000000,0.008403,0.336134,Bangalore Blockchain Summit happening at Allia...,"[bangalore, blockchain, summit, happening, all...","[(bangalore, RB), (blockchain, JJ), (summit, N...","[summit, alliance, university, panel, discussi..."
7,alliance university in partnership with kpi co...,TotalBitcoinInternational,1154961017325203456,2019-07-27 03:45:39,['Blockchain'],bitcoin_total,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,Alliance University in partnership with KPI Co...,"[alliance, university, partnership, kpi, consu...","[(alliance, NN), (university, NN), (partnershi...","[alliance, university, partnership, kpi, summi..."
8,on my way to our second @circleu_eu student s...,Youcef KADRI,1202917705046736896,2019-12-06 11:48:24,[],Youcef__KADRI,0.0,8.0,NaN,0.8614,Positive,5,0.000000,0.067227,2.689076,On my way to our second @CircleU_eu student se...,"[way, second, circleu_eu, student, seminar, lo...","[(way, NN), (second, JJ), (circleu_eu, JJ), (s...","[way, student, seminar, time, exchanges, europ..."
9,on my way to our second @circleu_eu student s...,Youcef KADRI,1202917705046736896,2019-12-06 11:48:24,[],Youcef__KADRI,0.0,8.0,NaN,0.8614,Positive,5,0.000000,0.067227,2.689076,On my way to our second @CircleU_e

In [14]:
#Filter the noun list only to make the list of attributes 
noun_lst = df['nouns'].apply(pd.Series).stack().unique()

# # get the frequencies for the nouns:
noun_df= pd.DataFrame(noun_lst)
noun_df.columns=['nouns']
noun_df

,nouns
0,indiapostoffice
1,sir
2,delivery
3,parcel
4,cu336449488in
5,pratham
6,agarwal
7,boys
8,alliance
9,university


In [15]:
## get frequency of all words
all_words = df['nouns'].sum()
freq_dist = nltk.FreqDist(all_words)

df_fdist=pd.DataFrame(list(freq_dist.items()), columns=['term', 'freq'])

noun_freq_df = pd.merge(noun_df, df_fdist, right_on='term', left_on='nouns')
noun_freq_df

,nouns,term,freq
0,indiapostoffice,indiapostoffice,2
1,sir,sir,8
2,delivery,delivery,2
3,parcel,parcel,4
4,cu336449488in,cu336449488in,2
5,pratham,pratham,2
6,agarwal,agarwal,2
7,boys,boys,2
8,alliance,alliance,555
9,university,university,579


In [16]:
noun_freq_df=noun_freq_df.sort_values(by=['freq'], ascending=False)
noun_freq_df

,nouns,term,freq
9,university,university,579
8,alliance,alliance,555
26,com,com,211
25,twitter,twitter,211
103,pic,pic,103
24,allianceuniversity,allianceuniversity,95
54,chancellor,chancellor,76
18,summit,summit,68
16,law,law,68
49,murder,murder,66


In [17]:
noun_freq_df.to_csv('nouns_tweets.csv')

In [18]:
#list of irreleveant tweet topics
l=['murder','death','suspects']
df['irrelevance']=df['nouns'].apply(lambda x: len(set(x).intersection(l)))
df[df.irrelevance!=0]

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance
10,alliance university v-c ayyappa dore murder: b...,The News Minute,1190838290712129536,2019-11-03 03:49:07,[],thenewsminute,0.0,1.0,NaN,-0.8357,Negative,1,0.000000,0.008403,0.336134,Alliance University V-C Ayyappa Dore murder Be...,"[alliance, university, v, c, ayyappa, dore, mu...","[(alliance, NN), (university, NN), (v, NN), (c...","[alliance, university, v, c, ayyappa, dore, mu...",1
13,alliance university v-c ayyappa dore murder: b...,The News Minute,1190838290712129536,2019-11-03 03:49:07,[],thenewsminute,0.0,1.0,NaN,-0.8357,Negative,1,0.000000,0.008403,0.336134,Alliance University V-C Ayyappa Dore murder Be...,"[alliance, university, v, c, ayyappa, dore, mu...","[(alliance, NN), (university, NN), (v, NN), (c...","[alliance, university, v, c, ayyappa, dore, mu...",1
219,former vc of alliance university ayyappa dore ...,IBTimes 🇮🇳,1184460232266833920,2019-10-16 13:25:00,"['AllianceUniversity', 'Bengaluruhttps']",ibtimes_india,1.0,1.0,NaN,-0.7650,Negative,1,0.047619,0.008403,3.193277,Former VC of Alliance University Ayyappa Dore ...,"[former, vc, alliance, university, ayyappa, do...","[(former, JJ), (vc, NN), (alliance, NN), (univ...","[vc, alliance, university, ayyappa, dore, deat...",1
220,"dr ayyappa, former vc of alliance university, ...",coastaldigest.com,1184453099370491904,2019-10-16 12:56:39,[],coastaldigest,1.0,0.0,NaN,-0.7650,Negative,1,0.047619,0.000000,2.857143,Dr Ayyappa former VC of Alliance University ha...,"[dr, ayyappa, former, vc, alliance, university...","[(dr, NN), (ayyappa, VBZ), (former, JJ), (vc, ...","[dr, vc, alliance, university, death]",1
221,former vc of alliance university hacked to dea...,Deccan Herald,1184444032602165248,2019-10-16 12:20:37,[],DeccanHerald,0.0,1.0,NaN,-0.7650,Negative,1,0.000000,0.008403,0.336134,Former VC of Alliance University hacked to dea...,"[former, vc, alliance, university, hacked, dea...","[(former, JJ), (vc, NN), (alliance, NN), (univ...","[vc, alliance, university, death, hmt, ground,...",1
223,former alliance university vc hacked to death ...,Digitalmunition,1184428903600922624,2019-10-16 11:20:30,[],maher275,0.0,0.0,NaN,-0.7650,Negative,1,0.000000,0.000000,0.000000,Former Alliance University VC hacked to death …,"[former, alliance, university, vc, hacked, death]","[(former, JJ), (alliance, NN), (university, NN...","[alliance, university, vc, death]",1
225,"ayyappa dore, former vc of alliance university...",The New Indian Express,1184412418514001920,2019-10-16 10:15:00,[],NewIndianXpress,0.0,3.0,NaN,-0.7650,Negative,1,0.000000,0.025210,1.008403,Ayyappa Dore former VC of Alliance University ...,"[ayyappa, dore, former, vc, alliance, universi...","[(ayyappa, RB), (dore, RB), (former, JJ), (vc,...","[vc, alliance, university, death, miscreants, ...",1
227,bengaluru police commissioner reacts over ex-v...,TV9 Kannada,1184395368961724416,2019-10-16 09:07:15,"['BengaluruPolice', 'AllianceUniversity', 'Mur...",tv9kannada,0.0,2.0,NaN,-0.6908,Negative,1,0.000000,0.016807,0.672269,Bengaluru Police Commissioner Reacts Over Ex-V...,"[bengaluru, police, commissioner, reacts, ex, ...","[(bengaluru, NN), (police, NN), (commissioner,...","[bengaluru, police, commissioner, vice, chance...",1
228,"retired vc of alliance university, ayyappa dor...",Chetana Belagere,1184377049756028928,2019-10-16 07:54:27,['AllianceUniversity'],chetanabelagere,5.0,5.0,NaN,-0.9423,Negative,1,0.238095,0.042017,15.966387,Retired VC of Alliance University Ayyappa Dore...,"[retired, vc, alliance, university, ayyappa, d...","[(retired, VBN), (vc, JJ), (alliance, NN), (un...","[alliance, university, ayyappa, dore, death, r...",1
231,former vc of alliance university ayyappa dore ...,Guest Post Expert,1184367011339624448,2019-10-16 07:14:34,[],szaheer123,0.0,0.0,NaN,-0.7650,Negative,1,0.000000,0.000000,0.000000,Former VC o

In [19]:
def assignpred(n):
    if n==2:
        return 1
    if n==4:
        return 5
    return n

In [20]:
engagement_median = df['engagement_score'].median()
df["engagement"] = df['engagement_score'].apply(lambda x: 1 if x > engagement_median else 0)

In [21]:
df['new_score']=df['score']

In [22]:
df_eng1=df[df.engagement==1]

In [23]:
df_eng1['new_score']=df_eng1['score'].apply(lambda x: assignpred(x))

/Users/Manam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
df[df.engagement==1]=df_eng1

In [25]:
#if the tweet is by university, assign sentiment as neutral
df.loc[df['uni_handle']==1,'new_score']=3

In [26]:
#if the tweet contains irrelevant terms, assign sentiment as neutral
df.loc[df['irrelevance']!=0,'new_score']=3

In [27]:
df.rename(columns = {'new_score':'sentiment_score'}, inplace = True) 
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,...,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance,engagement,sentiment_score
0,"@rsprasad @indiapostoffice \nsir,\nregarding u...",ranjeeta,1157270873314693120,2019-08-02 12:44:11,[],ranjeeta21,0.0,0.0,NaN,-0.2824,...,0.000000,0.000000,0.000000,"@rsprasad @IndiaPostOffice Sir, Regarding Urge...","[rsprasad, indiapostoffice, sir, regarding, ur...","[(rsprasad, JJ), (indiapostoffice, NN), (sir, ...","[indiapostoffice, sir, delivery, parcel, cu336...",0,0,2
1,edukhoj connected another student with school ...,EduKhoj,1156079145270439936,2019-07-30 05:48:41,['Bengaluru'],EduKhoj,0.0,0.0,NaN,0.0000,...,0.000000,0.000000,0.000000,EduKhoj connected another student with School ...,"[edukhoj, connected, another, student, school,...","[(edukhoj, RB), (connected, VBN), (another, DT...","[student, school, law, alliance, university, b...",0,0,3
2,bangalore blockchain summit happening at allia...,WWCode Blockchain,1155059429055709184,2019-07-27 10:16:42,"['blockchain', 'summit', 'allianceuniversity',...",WWCodeBlock,0.0,1.0,NaN,0.4588,...,0.000000,0.008403,0.336134,Bangalore Blockchain Summit happening at Allia...,"[bangalore, blockchain, summit, happening, all...","[(bangalore, RB), (blockchain, JJ), (summit, N...","[summit, alliance, university, panel, discussi...",0,1,5
3,alliance university in partnership with kpi co...,TotalBitcoinInternational,1154961017325203456,2019-07-27 03:45:39,['Blockchain'],bitcoin_total,0.0,0.0,NaN,0.0000,...,0.000000,0.000000,0.000000,Alliance University in partnership with KPI Co...,"[alliance, university, partnership, kpi, consu...","[(alliance, NN), (university, NN), (partnershi...","[alliance, university, partnership, kpi, summi...",0,0,3
4,"@rsprasad @indiapostoffice \nsir,\nregarding u...",ranjeeta,1157270873314693120,2019-08-02 12:44:11,[],ranjeeta21,0.0,0.0,NaN,-0.2824,...,0.000000,0.000000,0.000000,"@rsprasad @IndiaPostOffice Sir, Regarding Urge...","[rsprasad, indiapostoffice, sir, regarding, ur...","[(rsprasad, JJ), (indiapostoffice, NN), (sir, ...","[indiapostoffice, sir, delivery, parcel, cu336...",0,0,2
5,edukhoj connected another student with school ...,EduKhoj,1156079145270439936,2019-07-30 05:48:41,['Bengaluru'],EduKhoj,0.0,0.0,NaN,0.0000,...,0.000000,0.000000,0.000000,EduKhoj connected another student with School ...,"[edukhoj, connected, another, student, school,...","[(edukhoj, RB), (connected, VBN), (another, DT...","[student, school, law, alliance, university, b...",0,0,3
6,bangalore blockchain summit happening at allia...,WWCode Blockchain,1155059429055709184,2019-07-27 10:16:42,"['blockchain', 'summit', 'allianceuniversity',...",WWCodeBlock,0.0,1.0,NaN,0.4588,...,0.000000,0.008403,0.336134,Bangalore Blockchain Summit happening at Allia...,"[bangalore, blockchain, summit, happening, all...","[(bangalore, RB), (blockchain, JJ), (summit, N...","[summit, alliance, university, panel, discussi...",0,1,5
7,alliance university in partnership with kpi co...,TotalBitcoinInternational,1154961017325203456,2019-07-27 03:45:39,['Blockchain'],bitcoin_total,0.0,0.0,NaN,0.0000,...,0.000000,0.000000,0.000000,Alliance University in partnership with KPI Co...,"[alliance, university, partnership, kpi, consu...","[(alliance, NN), (university, NN), (partnershi...","[alliance, university, partnership, kpi, summi...",0,0,3
8,on my way to our second @circleu_eu student s...,Youcef KADRI,1202917705046736896,2019-12-06 11:48:24,[],Youcef__KADRI,0.0,8.0,NaN,0.8614,...,0.000000,0.067227,2.689076,On my way to our second @CircleU_eu student se...,"[way, second, circleu_eu, student, seminar, lo...","[(way, NN), (second, JJ), (circleu_eu, JJ), (s...","[way, student, seminar, time, exchanges, europ...",0,1,5
9,on my way to our second @circleu_eu student s...,Youcef KADRI,1202917705046736896,2019-12-06 11:48:24,[],Youcef__KADRI,0.0,8.0,NaN,0.8614,...,0.000000,0.067227,2.689076,On my way to our se

In [28]:
df.to_csv('../../Datasets/Twitter/Weighted Sentiment/AUWeightedSentiment.csv')